# MVTS Data Toolkit

## Demo

This demo is designed to give the user a quick tour over the software's funcionalities. Below is a list of all the things one could see in this demo:
 - Downloading a dataset of 2000 multivariate time series (mvts) instances.
 - Getting some basic statistics about your data.
 - Extracting a list of statistical features from the mvts instances.
 - ...

In [1]:
import os
import yaml
from data_retriever import DataRetriever  # for downloading data
import CONSTANTS as CONST

## Download the Dataset
In this demo we use an example dataset. In the following cells, it will be automatically downloaded. But in case something goes wrong, here is the direct link:
https://bitbucket.org/gsudmlab/mvtsdata_toolkit/downloads/petdataset_01.zip

Before we download it, let's take a quick look:

In [2]:
dr = DataRetriever(1)
dr.print_info()

URL:		https://bitbucket.org/gsudmlab/mvtsdata_toolkit/downloads/petdataset_01.zip
NAME:		petdataset_01.zip
TYPE:		application/zip
SIZE:		32M


Ready to download? This may take a few seconds, depending on your internet bandwidth. Wait for the progress bar.

In [3]:
where_to = './temp/'
dr.retrieve(target_path = where_to)

Extracting: 100%|██████████| 2001/2001 [00:00<00:00, 2948.21it/s]


OK. Let's see how many files are available to us now.

In [4]:
dr.get_total_number_of_files()

2000

## Setup Configurations
For tasks such as feature extraction (by `features.feature_extractor`) and data analysis (by `data_analysis.mvts_data_analysis` and `data_analysis.extracted_features_analysis`) a configuration file must be provided. There is one already provided inside this package, but you can create your own and place it anywhere you wish. Let's take a look at ours which is located at `./configs/feature_extraction_configs.yml`:

In [5]:
path_to_config = './configs/demo_configs.yml'
with open(path_to_config, 'r') as f:
    print(f.read())

PATH_TO_MVTS: './temp/petdataset_01/'
PATH_TO_EXTRACTED_FEATURES: './temp/extracted_features/'
META_DATA_TAGS: ['id', 'lab', 'st', 'et']
MVTS_PARAMETERS:
  - 'TOTUSJH'
  - 'TOTBSQ'
  - 'TOTPOT'
  - 'TOTUSJZ'
  - 'ABSNJZH'
  - 'SAVNCPP'
STATISTICAL_FEATURES:
  - 'get_min'
  - 'get_max'
  - 'get_median'
  - 'get_mean'
  - 'get_stddev'
  - 'get_var'
  - 'get_skewness'
  - 'get_kurtosis'
  - 'get_no_local_maxima'
  - 'get_no_local_minima'
  - 'get_no_local_extrema'
  - 'get_no_zero_crossings'
  - 'get_mean_local_maxima_value'
  - 'get_mean_local_minima_value'
  - 'get_no_mean_local_maxima_upsurges'
  - 'get_no_mean_local_minima_downslides'
  - 'get_difference_of_mins'
  - 'get_difference_of_maxs'
  - 'get_difference_of_means'
  - 'get_difference_of_stds'
  - 'get_difference_of_vars'
  - 'get_difference_of_medians'
  - 'get_dderivative_mean'
  - 'get_gderivative_mean'
  - 'get_dderivative_stddev'
  - 'get_gderivative_stddev'
  - 'get_dderivative_skewness'
  - 'get_gderivative_skewness'
  - 

Here is the break-down of the pieces:
 - `PATH_to_MVTS`: A relative or absolute path to where the multivariate time series dataset is stored at.
 - `PATH_TO_EXTRACTED`: A relative or absolute path to where the extracted features should be stored at, using the Feature Extraction component of the package.
 - `META_DATA_TAGS`: A list of tags based on which some pieces of information can be extracted from the file-names of the multivariate time series. For example, if timestamps are encoded in the file-names, e.g., `_st[YYYY-MM-DD HH:MM:SS]`, then the string `st` (without brackets) is a tag that can be included in this list. In the feature extraction process, this adds what is wrapped in the square brackets in each filename, as an extra column to the data-frame of the extracted features. Generally, using this functionality, any extra metadata can be encoded in the file-names and consequently passed into the extracted features.
 - `MVTS_PARAMETERS`: A list of parameter-names that are used in the multivariate time series dataset and their statistical features are of interest. These are, in other words, the column-names in the multivariate time series files.
 - `STATISTICAL_FEATURES`: A list of statistical features of interest to be extracted from the multivariate time series. They must be chosen from the provided methods in the module `features.feature_collection.py`. For example, `get_min` is a valid feature-name as this method is implemented in the package.
 
 In the following cells, you will see how this can be used.

## Analysis of Raw Data (MVTS Data Analysis)

- #### How many files? How large of a dataset?

Using `mvts_data_analysis` we can get an idea of the dataset we are going to work on. We start with creating an instance of a `MVTSDataAnalysis`.

In [6]:
from data_analysis.mvts_data_analysis import MVTSDataAnalysis
path_to_config = './configs/demo_configs.yml'
mvda = MVTSDataAnalysis(path_to_config)
mvda.print_stat_of_directory()

----------------------------------------
Directory:			/home/azim/CODES/PyCode/mvtsdata_toolkit/./temp/petdataset_01/
Total no. of files:	2000
Total size:			76M
Total average:		38K
----------------------------------------


- #### Get a summary stats of the data.

Let's now get some statistics from the content of the files. To speed up the demo, we analyze only 3 parameters (namely `TOTUSJH`, `TOTBSQ`, and `TOTPOT`), and only the first 50 mvts files.

In [7]:
params = ['TOTUSJH', 'TOTBSQ', 'TOTPOT']
n = 50
mvda.compute_summary(params_name=params, first_k=n)
mvda.summary

,Parameter-Name,Val-Count,Null-Count,mean,min,25th,50th,75th,max
0,TOTUSJH,3000,0,8.042049e+02,4.341081e+00,5.959445e+01,3.416911e+02,1.112788e+03,3.746423e+03
1,TOTBSQ,3000,0,1.149033e+10,2.376532e+07,4.003989e+08,4.548718e+09,1.446302e+10,6.957558e+10
2,TOTPOT,3000,0,1.871504e+23,1.750878e+20,2.813785e+21,3.847445e+22,2.628458e+23,1.425084e+24


... which says the length of the time series, across the 50 mvts files is 3000, including 11 `NA/NAN` values. In addition, `mean`, `min`, `max`, and three quantiles are calculated for each time series.

 - #### You have a LARGE dataset?
 A parallel version of this function is also provided to help process much larger datasets efficiently. Below, we will have 4 processes to compute the summary statistics.

In [8]:
mvda.compute_summary_in_parallel(n_jobs=4, first_k=50, verbose=False,
                                     params_name=['TOTUSJH', 'TOTBSQ', 'TOTPOT'])
mvda.summary

,Parameter-Name,Val-Count,Null-Count,mean,min,25th,50th,75th,max
0,TOTUSJH,3000,0,7.996792e+02,4.341081e+00,5.958282e+01,3.506740e+02,1.121246e+03,3.746423e+03
1,TOTBSQ,3000,0,1.140743e+10,2.376532e+07,4.004747e+08,4.554687e+09,1.448875e+10,6.957558e+10
2,TOTPOT,3000,0,1.855639e+23,1.750878e+20,2.814335e+21,3.832085e+22,2.628251e+23,1.425084e+24


**Note**: The results of the parallel and sequential versions of `mvts_data_analysis` are not exactly identical. This discrepency is due to the fact that in the parallel version, the program approximates the percentiles. more specifically, it is designed to avoid loading the entire dataset into memory so that it is not confined to any particular data size. Therefore, it relies on some statistical estimators to approximate the percentiles with some acceptable errors. The errors decrease significantly as the number of mvts files increases.

## Feature Extraction

- #### What statistical features are available?

Now that we have an idea about our raw data, let's extract some features from the data. A list of ~50 statistical features are implemented in `feature_collection`. Let's take a look at them.

In [9]:
import features.feature_collection as fc
help(fc)

Help on module features.feature_collection in features:

NAME
    features.feature_collection

FUNCTIONS
    get_average_absolute_change(uni_ts:Union[pandas.core.series.Series, numpy.ndarray]) -> numpy.float64
        :return: the average absolute first difference of a univariate time series.
    
    get_average_absolute_derivative_change(uni_ts:Union[pandas.core.series.Series, numpy.ndarray]) -> numpy.float64
        :return: the average absolute first difference of a derivative of univariate time series.
    
    get_avg_mono_decrease_slope(uni_ts:Union[pandas.core.series.Series, numpy.ndarray]) -> numpy.float64
        :return: the average slope of monotonically decreasing segments.
    
    get_avg_mono_increase_slope(uni_ts:Union[pandas.core.series.Series, numpy.ndarray]) -> numpy.float64
        :return: the average slope of monotonically increasing segments.
    
    get_dderivative_kurtosis(uni_ts:Union[pandas.core.series.Series, numpy.ndarray], step_size:int=1) -> numpy.float

- #### How to extract these features from the data?

Let's extract 3 simple statistical features, namely `min`, `max`, and `median`, from 3 parameters, such as `TOTUSJH`, `TOTBSQ`, and `TOTPOT`. Again, to speed up the process in this demo, we only process the first 50 mvts files.

In [10]:
from features.feature_extractor import FeatureExtractor

fe = FeatureExtractor(path_to_config)
fe.do_extraction(features_name=['get_min', 'get_max', 'get_median'],
                 params_name=['TOTUSJH', 'TOTBSQ', 'TOTPOT'], first_k=50)
fe.df_all_features

,id,lab,st,et,TOTUSJH_min,TOTUSJH_max,TOTUSJH_median,TOTBSQ_min,TOTBSQ_max,TOTBSQ_median,TOTPOT_min,TOTPOT_max,TOTPOT_median
0,3894,M,2014-03-29T22:00:00,2014-03-30T09:48:00,967.434905,1183.833149,1124.410250,1.046786e+10,1.188663e+10,1.153321e+10,1.228731e+23,1.716971e+23,1.492330e+23
1,3364,NF,2013-11-10T16:24:00,2013-11-11T04:12:00,785.515878,898.419700,841.636672,1.384462e+10,1.505713e+10,1.449548e+10,2.558560e+23,2.903506e+23,2.712413e+23
2,3401,NF,2013-11-19T15:00:00,2013-11-20T02:48:00,43.795702,79.405460,61.347579,3.006945e+08,4.249081e+08,3.883491e+08,1.854126e+21,3.078696e+21,2.787184e+21
3,3595,NF,2014-01-12T09:24:00,2014-01-12T21:12:00,48.647340,65.213699,54.220981,3.276020e+08,4.447089e+08,3.539810e+08,2.278193e+21,3.308322e+21,2.707839e+21
4,3452,NF,2013-12-06T18:24:00,2013-12-07T06:12:00,54.910546,89.248031,64.519227,3.395135e+08,6.300441e+08,4.605988e+08,2.030967e+21,4.775595e+21,2.968826e+21
5,3401,NF,2013-11-23T00:00:00,2013-11-23T11:48:00,16.406938,25.926148,20.134504,1.098692e+08,1.766917e+08,1.430579e+08,6.961236e+20,1.368650e+21,9.542322e+20
6,3313,NF,2013-10-29T06:48:00,2013-10-29T18:36:00,25.213735,45.237145,37.858018,1.640152e+08,3.040086e+08,2.364114e+08,1.079530e+21,2.178500e+21,1.615138e+21
7,3394,NF,2013-11-16T12:48:00,2013-11-17T00:36:00,37.198119,105.665566,74.004088,4.422474e+08,1.066918e+09,7.937949e+08,2.419303e+21,6.886243e+21,4.586577e+21
8,3779,C,2014-02-26T06:24:00,2014-02-26T18:12:00,761.638915,987.804217,901.833056,5.706467e+09,7.378234e+09,6.651778e+09,5.153352e+22,7.974288e+22,7.048366e+22
9,3364,C,2013-11-17T20:24:00,2013-11-18T08:12:00,3410.057526,3746.423110,3583.958457,4.832398e+10,5.258952e+10,5.117475e+10,7.566750e+23,8.037784e+23,7.870867e+23


... where each row corresponds to one mvts file, and the first 4 columns represent the extracted information from the file-names using the tags specified in the configuration file (i.e., `id`, `lab`, `st`, and `et`).

 - #### You have a LARGE dataset?
 No worries. Using the parallel implementation of feature extraction, the process can be significantly sped up. Let's have 4 processes to extract the features.

In [11]:
fe.do_extraction_in_parallel(n_jobs=4,
                             features_name=['get_min', 'get_max', 'get_median'],
                             params_name=['TOTUSJH', 'TOTBSQ', 'TOTPOT'], first_k=50)
fe.df_all_features

,id,lab,st,et,TOTUSJH_min,TOTUSJH_max,TOTUSJH_median,TOTBSQ_min,TOTBSQ_max,TOTBSQ_median,TOTPOT_min,TOTPOT_max,TOTPOT_median
0,3513,NF,2013-12-21T04:24:00,2013-12-21T16:12:00,246.640181,379.515997,295.875850,1.647788e+09,2.648920e+09,2.038514e+09,1.194453e+22,2.124787e+22,1.629861e+22
1,3295,M,2013-10-27T16:00:00,2013-10-28T03:48:00,2276.003929,2486.069447,2336.941111,3.669545e+10,3.846283e+10,3.718973e+10,5.947570e+23,6.569605e+23,6.192929e+23
2,3448,NF,2013-12-03T07:24:00,2013-12-03T19:12:00,627.381008,749.396580,685.068657,4.579947e+09,5.401685e+09,4.843743e+09,3.518158e+22,4.252477e+22,3.870690e+22
3,3364,X,2013-11-18T11:24:00,2013-11-18T23:12:00,3112.763415,3636.879129,3398.117993,5.000181e+10,5.547602e+10,5.401976e+10,7.127595e+23,7.943010e+23,7.474336e+23
4,3515,NF,2013-12-20T08:24:00,2013-12-20T20:12:00,475.294739,571.634279,531.393544,5.668066e+09,6.700476e+09,6.249927e+09,9.619054e+22,1.088221e+23,1.042996e+23
5,3576,NF,2014-01-09T10:12:00,2014-01-09T22:00:00,20.002664,28.046892,24.012224,1.864957e+08,2.450552e+08,2.083817e+08,1.208031e+21,1.709355e+21,1.424422e+21
6,3371,NF,2013-11-15T21:12:00,2013-11-16T09:00:00,355.494117,462.495793,405.551996,5.210313e+09,6.769616e+09,6.045823e+09,4.783879e+22,6.282941e+22,5.326339e+22
7,3492,NF,2013-12-10T04:00:00,2013-12-10T15:48:00,79.752980,164.666575,137.614010,5.709126e+08,1.810153e+09,1.369314e+09,5.440338e+21,1.515428e+22,1.282723e+22
8,3561,NF,2014-01-01T01:12:00,2014-01-01T13:00:00,67.218761,98.099020,86.452392,5.940839e+08,1.013197e+09,8.182449e+08,4.758672e+21,6.611339e+21,5.644317e+21
9,3370,NF,2013-11-15T09:00:00,2013-11-15T20:48:00,20.865132,40.132998,31.198306,1.520483e+08,2.684384e+08,2.176681e+08,9.282716e+20,1.892371e+21,1.496236e+21


**Note**: Due to the use of multiprocessing, the two outputs (using either single or multiple processes) are only comparable when a unqiue column (here `id`) is set as index.

## Extracted Features Analysis

- #### A quick look over the results?

The extracted features can be easily summarized using descriptive statistics such as `meam`, `std`, `min`, `max`, and first, second and third quartiles. In addition, any missing value can also be spotted.

In [12]:
from data_analysis.extracted_features_analysis import ExtractedFeaturesAnalysis

efa = ExtractedFeaturesAnalysis(fe.df_all_features, exclude=['id'])
efa.compute_summary()
efa.summary

,Feature-Name,Val-Count,Null-Count,mean,std,min,25th,50th,75th,max
0,TOTUSJH_min,50.0,0,7.341838e+02,9.667145e+02,4.341081e+00,4.879846e+01,3.144178e+02,9.276320e+02,3.410058e+03
1,TOTUSJH_max,50.0,0,8.746654e+02,1.112331e+03,1.226119e+01,7.940546e+01,3.981758e+02,1.137049e+03,3.746423e+03
2,TOTUSJH_median,50.0,0,8.033427e+02,1.041983e+03,7.426589e+00,6.063404e+01,3.522123e+02,1.068766e+03,3.583958e+03
3,TOTBSQ_min,50.0,0,1.076656e+10,1.574154e+10,2.376532e+07,3.388376e+08,4.200314e+09,1.340520e+10,5.766012e+10
4,TOTBSQ_max,50.0,0,1.210067e+10,1.744910e+10,6.548704e+07,4.603817e+08,4.922122e+09,1.470204e+10,6.957558e+10
5,TOTBSQ_median,50.0,0,1.158519e+10,1.697273e+10,3.557751e+07,4.064115e+08,4.513584e+09,1.409818e+10,6.751291e+10
6,TOTPOT_min,50.0,0,1.778611e+23,2.921604e+23,1.750878e+20,2.100021e+21,3.426977e+22,2.431402e+23,1.369838e+24
7,TOTPOT_max,50.0,0,1.971327e+23,3.104716e+23,6.645951e+20,3.355905e+21,4.175589e+22,2.739604e+23,1.425084e+24
8,TOTPOT_median,50.0,0,1.872583e+23,3.006618e+23,3.158317e+20,2.797750e+21,3.834246e+22,2.577412e+23,1.398754e+24


... which gives a summary statistics over every extracted feature. For instance, in row `0`, the changes of the minimum values of the parameter `TUOTUSJH`, across 50 mvts files, is described in terms of `mean`, `std`, etc. This also indicates that no `NA/NAN` or missing value was generated in the process.

## Data Normalization

The extracted features can then be normalized using four different methods. Below, you see a zero-one normalization being used.

In [13]:
from normalizing import normalizer

df_norm = normalizer.zero_one_normalize(df=fe.df_all_features, excluded_colnames=['id'])
df_norm

,id,lab,st,et,TOTUSJH_min,TOTUSJH_max,TOTUSJH_median,TOTBSQ_min,TOTBSQ_max,TOTBSQ_median,TOTPOT_min,TOTPOT_max,TOTPOT_median
0,3513,NF,2013-12-21T04:24:00,2013-12-21T16:12:00,0.071145,0.098350,0.080651,0.028177,0.037166,0.029683,0.008593,0.014450,0.011429
1,3295,M,2013-10-27T16:00:00,2013-10-28T03:48:00,0.667015,0.662480,0.651333,0.636260,0.552399,0.550617,0.434108,0.460746,0.442620
2,3448,NF,2013-12-03T07:24:00,2013-12-03T19:12:00,0.182939,0.197403,0.189469,0.079050,0.076769,0.071256,0.025558,0.029388,0.027453
3,3364,X,2013-11-18T11:24:00,2013-11-18T23:12:00,0.912707,0.970664,0.948039,0.867127,0.797158,0.800034,0.520263,0.557165,0.534252
4,3515,NF,2013-12-20T08:24:00,2013-12-20T20:12:00,0.138283,0.149799,0.146501,0.097930,0.095454,0.092095,0.070102,0.075931,0.074357
5,3576,NF,2014-01-09T10:12:00,2014-01-09T22:00:00,0.004599,0.004227,0.004637,0.002823,0.002583,0.002561,0.000754,0.000733,0.000793
6,3371,NF,2013-11-15T21:12:00,2013-11-16T09:00:00,0.103107,0.120572,0.111316,0.089987,0.096448,0.089071,0.034800,0.043642,0.037862
7,3492,NF,2013-12-10T04:00:00,2013-12-10T15:48:00,0.022143,0.040814,0.036400,0.009493,0.025099,0.019766,0.003844,0.010172,0.008947
8,3561,NF,2014-01-01T01:12:00,2014-01-01T13:00:00,0.018462,0.022987,0.022096,0.009895,0.013634,0.011599,0.003347,0.004175,0.003810
9,3370,NF,2013-11-15T09:00:00,2013-11-15T20:48:00,0.004852,0.007464,0.006647,0.002226,0.002920,0.002699,0.000550,0.000862,0.000844


**Note**: The argument `excluded_colnames` is used to keep the column `id` intact in the normalization process. Although this column is numeric, normalization of its values would be meaningless. Moreover, any other column with non-numeric values were automatically preserved in the output.

In [ ]:
from visualizations.stat_visualizer import StatVisualizer

StatVisualizer()